In [9]:
import sys
import os
os.environ[
    "REDIS_OM_URL"
] = "redis://:aclkasjf29qwrUOIO@tiger.lti.cs.cmu.edu:6388"

import json
from tqdm.notebook import tqdm
import rich
import logging
from collections import defaultdict, Counter
from sotopia.database.persistent_profile import AgentProfile, EnvironmentProfile, RelationshipProfile
from sotopia.database.logs import EpisodeLog
from sotopia.database.env_agent_combo_storage import EnvAgentComboStorage
from collections import Counter 
from redis_om import Migrator
from rich.console import Console
from rich.terminal_theme import MONOKAI 
import matplotlib.pyplot as plt
import numpy as np
from prompt_reverse_engineering import reverse_episode_log, parse_prompt_to_json
from redis_filtering import get_clean_episodes, align_episode_by_env, goal_filter_all_env_agent, run_filtered_episodes_to_prompt, goal_reward_by_env_agent


In [ ]:
SELECTED_TAG = ["ft-mistral-7b-old-filtererd-data_gpt-3.5_clean_ruiyi_1116"] # "gpt-4_gpt-4_v0.0.1_clean", 
# difficulty could vary by agent - same scenario might be hard for one and normal for the other
# for our purpose, it is fair to use dialogue, regardless of agents, in hard testing set 
HARD_SCENARIO = set(['01H7VFHNV13MHN97GAH73E3KM8', '01H7VFHN5WVC5HKKVBHZBA553R', '01H7VFHNN7XTR99319DS8KZCQM', 
                 '01H7VFHN9W0WAFZCBT09PKJJNK', '01H7VFHPDZVVCDZR3AARA547CY', '01H7VFHPQQQY6H4DNC6NBQ8XTG', 
                 '01H7VFHPQQQY6H4DNC6NBQ8XTG', '01H7VFHN7WJK7VWVRZZTQ6DX9T', '01H7VFHN7A1ZX5KSMT2YN9RXC4', 
                 '01H7VFHPS5WJW2694R1MNC8JFY', '01H7VFHPS5WJW2694R1MNC8JFY', '01H7VFHNN7XTR99319DS8KZCQM', 
                 '01H7VFHQ11NAMZS4A2RDGDB01V', '01H7VFHQ11NAMZS4A2RDGDB01V', '01H7VFHPSWGDGEYRP63H2DJKV0', 
                 '01H7VFHPSWGDGEYRP63H2DJKV0', '01H7VFHNF4G18PC9JHGRC8A1R6', '01H7VFHNNYH3W0VRWVY178K2TK', 
                 '01H7VFHP8AN5643B0NR0NP00VE', '01H7VFHN7A1ZX5KSMT2YN9RXC4'])


episode_by_tag = get_clean_episodes(SELECTED_TAG)
concat_epilist = sum(episode_by_tag.values(), [])
dic_epi_env = align_episode_by_env(concat_epilist)
filter_agent_episodes = goal_filter_all_env_agent(dic_epi_env)

In [3]:
def get_episodes_by_env_dict(
    tags: list, 
    scenarios: set,
) -> dict[str, list[EpisodeLog]]:
     # Load all episodes from tags
    eps_by_tag = {}
    for tag in tags:
        eps = EpisodeLog.find(EpisodeLog.tag == tag).all()
        if len(eps) > 0:
            eps_by_tag[tag] = eps
    eps_list = sum(eps_by_tag.values(), [])

    # Only select episodes under target scenarios
    eps_by_env = {}
    for ep in eps_list:
        if ep.environment in scenarios:
            if ep.environment in eps_by_env:
                eps_by_env[ep.environment].append(ep)
            else:
                eps_by_env[ep.environment] = [ep]
    
    return eps_by_env


def get_sorted_episode_list_for_target_agent(
    tags: list, 
    scenarios: set,
    agent_model: str,
    reward_metric: str = "overall_score"
) -> list:
    
    eps_by_env = get_episodes_by_env_dict(tags, scenarios)
    eps_list = sum(eps_by_env.values(), [])
    
    # Split into two cases where the target agent converses with another agent or itself
    single_agent_eps_list = []
    dual_agent_1_eps_list = []
    dual_agent_2_eps_list = []
    for ep in eps_list:
        if ep.models[1] == agent_model and ep.models[2] == agent_model:
            dual_agent_1_eps_list.append((1, ep))
            dual_agent_2_eps_list.append((2, ep))
        elif ep.models[1] == agent_model or ep.models[2] == agent_model:
            single_agent_eps_list.append((0, ep))

    combined_eps_list = single_agent_eps_list + dual_agent_1_eps_list + dual_agent_2_eps_list

    def reward_sort_fn(x):
        if x[0] == 0:
            return x[1].rewards[0][1][reward_metric] if x[1].models[1] == agent_model else x[1].rewards[1][1][reward_metric]
        elif x[0] == 1:
            return x[1].rewards[0][1][reward_metric]
        else:
            return x[1].rewards[1][1][reward_metric]
        
    sorted_combined_eps_list = sorted(combined_eps_list, key=reward_sort_fn, reverse=True)

    return sorted_combined_eps_list


def select_top_reward_eps(
    sorted_eps_list: list,
    ratio: float = 0.7
) -> list:
    return sorted_eps_list[:round(ratio * len(sorted_eps_list))]

In [5]:
TAGS = ["init_selftrain_round_1_mistral_gpt-4_clean", "init_selftrain_round_1_mistral_gpt-3.5-turbo_clean", "init_selftrain_round_1_mistral_mistral_clean"]
TARGET_AGENT = "custom_model"
with open("../../data_generate/env_files/used_env.json", 'r') as f:
    SCENARIOS = json.loads(f.read())["SFT-round-1"]
# SCENARIOS = ["01HFSDNWGEK1808TP4ZDVZCJEP", "01HFSDNWGHBV0SDXS9FT3P6AF9", "01HFSDNWG8RG3XNA8TNHE00PWD", "01HFSDNWG9NFCGCX3DZ39HHBNR", "01HFSDNWFYTK35BFQDGHTJYBAG", "01HFSDNWHKRS2E646B4K9DQS8Z", "01HFSDNWH76WPTBG0BKRRAMNFG", "01HFSDNWF5X4NJQ6EZPKEDGVGX", "01HFSDNWH17A1ADSAM3BJGW914", "01HFSDNWFZKX4SVRWFDD4GKAMC", "01HFSDNWGBJ7HJXAJVPCZPS491", "01HFSDNWFWG5KJMZ80D10VBYQ7", "01HFSDNWGA8KXENGCJYPHD1YQW", "01HFSDNWFJGESH1WB5NYYEAST8", "01HFSDNWGTP2EMNM949JRA20H0", "01HFSDNWHA7X3EM8S0ZQF94TBH", "01HFSDNWFQ616QWSFSP4RDHJ7S", "01HFSDNWFCEK8NDHMT8EX88CMY", "01HFSDNWFX9NYHZ2GRVN2D1494", "01HFSDNWH0RZDXD8MRX0FVETWC"]

In [6]:
sorted_combined_eps_list = get_sorted_episode_list_for_target_agent(TAGS, SCENARIOS, TARGET_AGENT)

In [8]:
sorted_combined_eps_list[:10]

[(0,
  EpisodeLog(pk='01HMS1TBJW8XJ97YESV7Z904W3', environment='01HJPQ34NZYMHJ00EMBD6R3NJV', agents=['01H5TNE5P90FYSTBMW5DG5ERCG', '01H5TNE5PHQKQYWS9ZS2JVEYFS'], tag='init_selftrain_round_1_mistral_gpt-3.5-turbo_clean', models=['gpt-4', 'custom_model', 'gpt-3.5-turbo'], messages=[[('Environment', 'Isabella White', "\nHere is the context of this interaction:\nScenario: Agent1 is at a technology store seeking help from Agent2, a customer service representative, due to a misunderstanding in a previous communication that led to Agent1's device not being repaired as expected.\nParticipants: Isabella White and Lena Goodwin\nIsabella White's background: Isabella White is a 40-year-old female veterinarian. She/her pronouns. Isabella White, a devoted veterinarian, is well-loved in her community for her kindness towards animals. Personality and values description: Isabella White, introverted and conscientious, values benevolence. Her empathy dictates her decision-making process. Isabella's secre

"01HFSDNWG78KV60B0H60GQNNF9", "01HFSDNWGPJSBBS6HT8GDWMC9Q", "01HFSDNWG5Z99ZX40DHZY88PY8", "01HFSDNWHDNSAWC41SWM1ZXTCG", "01HFSDNWGMHY3VNT4BGDXP0GZN", "01HFSDNWG1WFWWBEMMQ0M694BG"

In [6]:
sorted_combined_eps_list = get_sorted_episode_list_for_target_agent(TAGS, SCENARIOS, TARGET_AGENT)
selected_eps_list = select_top_reward_eps(sorted_combined_eps_list, ratio=0.7)
print(len(selected_eps_list))

104


In [28]:
def get_train_data_from_eps_list(
    selected_eps_list: list,
    output_file: str
):
    conv_list = []
    for ep in selected_eps_list:
        if ep[0] == 0: # target agent + other agent
            if ep[1].models[1] == TARGET_AGENT:
                conv = reverse_episode_log(ep[1], later_speak=False)
            else:
                conv = reverse_episode_log(ep[1], later_speak=True)
        elif ep[0] == 1: # target agent * 2, higher speaks first
            conv = reverse_episode_log(ep[1], later_speak=False)
        else: # target agent * 2, higher speaks second
            conv = reverse_episode_log(ep[1], later_speak=True)
        
        for turn in conv:
            conv_list.append({
                "instruction": "",
                "input": turn['prompt'],
                "output": turn['result']
            })
    
    with open(output_file, 'w') as f:
        f.write(json.dumps(conv_list, indent=4))

In [29]:
get_train_data_from_eps_list(selected_eps_list, output_file="data_generation_20_scenarios.json")